# Download RGB

1 - General setting up

In [1]:
import os
os.environ.pop('PROJ_LIB', None)
os.environ.pop('GDAL_DATA', None)
# import libraries
import ee
import geemap
ee.Authenticate()
ee.Initialize()
# from rgb_func import*
# Download years
yrList = [2021,2022,2023,2024]
# Number of parallelization
cpus_ = 7

c:\Users\spn733\Miniconda3\envs\rgb-environment\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


2 - Select a province to download

In [2]:
from pathlib import Path

# Automatically find the project root (assumes this notebook is in 2_RGB_download)
project_root = Path.cwd().parent

def get_out_dir(provName):
    if provName == 'AB':
        return str(project_root / '5_Data' / 'RGB_download' / 'Alberta')
    elif provName == 'SK':
        return str(project_root / '5_Data' / 'RGB_download' / 'Saskatchewan')
    elif provName == 'MB':
        return str(project_root / '5_Data' / 'RGB_download' / 'Manitoba')
    else:
        raise ValueError("Invalid province name. Use 'AB', 'SK', or 'MB'.")

# Select province
provName = 'SK'  # AB, SK, MB
if provName == 'AB':
    prov = 'Alb.'
    asset_path = 'projects/ee-aafc-annimation/assets/alberta_highway'  # AB
elif provName == 'SK':
    prov = 'Sask.'
    asset_path = "projects/just-amp-296821/assets/road_sk_utm13"  # SK
elif provName == 'MB':
    prov = 'Man.'
    asset_path = 'projects/just-amp-296821/assets/road_mb_utm14'  # MB
else:
    raise ValueError("Invalid province name. Use 'AB', 'SK', or 'MB'.")

out_dir = get_out_dir(provName)

3 - Get tiles

In [3]:
allProvince = ee.FeatureCollection('projects/ee-aafc-annimation/assets/provincialBoundary')
pruid_list = allProvince.aggregate_array('PRFABBR')
selectProv = allProvince.filter(ee.Filter.stringStartsWith('PRFABBR', prov))
# selectProv


# Load full grid and convert to list
grid = ee.FeatureCollection('projects/ee-download-canada/assets/Grid_prairies').filterBounds(selectProv)
list_roi_all = grid.toList(grid.size())
grid_size = list_roi_all.size().getInfo()
# grid_size

4 - Setup downloader

In [4]:
import time
from multiprocessing import Pool
from tqdm import tqdm

# Define functions used in downloading RGB images

def get_s2(roi, year, mask,
           s2_collection='COPERNICUS/S2_SR_HARMONIZED',
           csplus_collection='GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED',
           qa_band='cs_cdf',
           clear_threshold=0.55):
    
    # Load collections
    s2 = ee.ImageCollection(s2_collection)
    csPlus = ee.ImageCollection(csplus_collection)

    # Define join function
    def join_cloudscore(s2_col, cs_col):
        join = ee.Join.saveFirst('csplus')
        filter = ee.Filter.equals(leftField='system:index', rightField='system:index')
        return ee.ImageCollection(join.apply(s2_col, cs_col, filter))

    # Define cloud masking function
    def apply_cloud_mask(image):
        cs_img = ee.Image(image.get('csplus')).select(qa_band)
        return image.updateMask(cs_img.gte(clear_threshold))

    # Filter collections
    s2_filtered = s2.filterBounds(roi).filterDate(f'{year}-04-01', f'{year}-10-01')
    cs_filtered = csPlus.filterBounds(roi).filterDate(f'{year}-04-01', f'{year}-10-01')

    # Join and apply cloud mask
    joined = join_cloudscore(s2_filtered, cs_filtered)
    collection = joined.map(apply_cloud_mask)

    # Extract and scale RGB bands
    B4 = (collection.filter(ee.Filter.calendarRange(8, 10, 'month'))
          .select('B4').median().unitScale(100, 3000).multiply(255).toByte().unmask(1).rename('B4'))
    

    B3 = (collection.filter(ee.Filter.calendarRange(6, 8, 'month'))
          .select('B3').median().unitScale(200, 2300).multiply(255).toByte().unmask(1).rename('B3'))

    B2 = (collection.filter(ee.Filter.calendarRange(4, 7, 'month'))
          .select('B2').median().unitScale(150, 1350).multiply(255).toByte().unmask(1).rename('B2'))


    # Combine and mask
    rgb = B4.addBands(B3).addBands(B2)
    rgb_3m = rgb.multiply(mask).toByte().unmask(1).clip(roi)

    return rgb_3m


def get_aafc_crop_mask(roi):
    """
    Generates a smoothed AAFC crop mask from the AAFC Crop Inventory.
    
    Parameters:
    - roi: ee.Geometry or ee.Feature to clip the output.
    - start_date: start of date filter (default '2015-01-01')
    - end_date: end of date filter (default '2024-12-31')
    
    Returns:
    - ee.Image: Smoothed binary crop mask image.
    """
    # time range
    start_date='2015-01-01'
    end_date='2024-12-31'
    
    # Load AAFC Crop Inventory collection
    crop = ee.ImageCollection('AAFC/ACI').filterDate(start_date, end_date)

    # Define remapping lists
    oldVal = [10, 20, 30, 34, 35, 50, 80, 110, 120, 122, 130, 131, 132, 133, 134, 135, 136, 137,
              138, 139, 140, 141, 142, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
              157, 158, 160, 162, 167, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 185, 188,
              189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 210, 220, 230]

    newVal = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 132, 133, 134, 135, 136, 137,
              138, 139, 140, 141, 142, 145, 146, 147, 148, 149, 150, 151, 152, 153,
              154, 155, 156, 157, 158, 160, 162, 167, 174, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

    # Define image mapping function
    def map_crop_mask(img):
        mask = img.remap(oldVal, newVal, 0, 'landcover').rename('landcover').toUint32()
        cropMask = mask.gt(1).clip(roi)
        return cropMask

    # Apply remapping and processing
    aafcMask = crop.map(map_crop_mask).sum().focal_mean(radius=3).rename('aafcMask').divide(10)
    return aafcMask.clip(roi)


# Define the combined function
def processCroplandMask(roi,roadMask):
    # Index calculation function
    def index(img):
        ndvi = img.normalizedDifference(['B8', 'B4']).rename('ndvi')
        ndwi = img.normalizedDifference(['B8', 'B11']).rename('ndwi')
        return img.addBands(ndvi).addBands(ndwi) \
            .copyProperties(img, ['system:time_start'])
    
    # Cropland filter function
    def croplandFilter(img):
        minArea = 50000
        maxSize = 200
        filter = img.focal_mean(2, 'square', 'pixels', 1).gt(0.5)
        pixelCount = filter.connectedPixelCount(maxSize)
        minPixelCount = ee.Image(minArea).divide(ee.Image.pixelArea())
        imgFiltered = filter.updateMask(pixelCount.gte(minPixelCount)).unmask()
        return imgFiltered
    
    # S2 collection
    s2 = ee.ImageCollection("COPERNICUS/S2") \
        .filterBounds(roi) \
        .filterDate('2022-04-01', '2023-10-15') \
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 5)) \
        .filter(ee.Filter.calendarRange(4, 9, 'month')) \
        .map(lambda img: img.clip(roi))
    
    # Extract indices
    NDVImax = s2.map(index).filter(ee.Filter.calendarRange(4, 9, 'month')).select('ndvi').max()
    NDVImedian = s2.map(index).filter(ee.Filter.calendarRange(4, 6, 'month')).select('ndvi').median()
    NDVIdiff = NDVImax.subtract(NDVImedian)
    NDWImin = s2.map(index).select('ndwi').min()
    
    # Non-crop index calculation
    nonCrop= NDVIdiff.subtract(NDWImin)
    testaafc = get_aafc_crop_mask(roi)
  
    croplandMask = nonCrop.multiply(testaafc).add(nonCrop).multiply(roadMask) #croplandFilter(nonCrop).clip(roi)
  
    return croplandMask.unmask(0.1)#filteredNonCroplandMask


# Make an image out of the 'val' attribute
def reduceToImage(featureCollection):
    return featureCollection\
        .filter(ee.Filter.notNull(['val']))\
        .reduceToImage(properties=['val'], reducer=ee.Reducer.first())


# Create a buffer around road features and set 'val' attribute to 1
def bufferAndSetVal(feature):
    return feature.buffer(5).set('val', 1)

def get_crp_rgb_from_asset(yrList, download_dir, provName, local_idx, asset_path, selectProv, tile_shp):
    
    for yr in yrList:

        import ee, os
        ee.Initialize()
        #import geemap.geemap as geemap
        from pathlib import Path
        
        # Set output file names
        output_tif = str(download_dir) + '/rgb_' + provName + '_' + str(yr) + '_' + str(local_idx) + '_1.tif'
        
        # if output csv file exist, skip
        if os.path.exists(output_tif):
            
            print("Crop RGB exists for file ",
                  str(Path(output_tif).stem), ", skipping now ...")
            
        else:

            # Road mask
            road_skshp = ee.FeatureCollection(asset_path).filterBounds(selectProv)
            mask = ee.Image.constant(1).clip(selectProv)
            roadSK = road_skshp.map(bufferAndSetVal)
            roadImg = reduceToImage(roadSK)
            roadMask = mask.multiply(roadImg.add(0.3)).mask(0.1).Not().clip(selectProv)

            # Filter for required year
            start_date =  str(yr) +'-05-15'
            end_date = str(yr)+'-10-15'
            year = str(yr)

            geo = ee.Feature(tile_shp).geometry()

            # MASK from NDVI and ESA
            mask = processCroplandMask(geo, roadMask)# using crop and road
            rgb_3m = get_s2(geo, yr, mask)

            geemap.download_ee_image_tiles(
                rgb_3m, ee.FeatureCollection(ee.Feature(tile_shp)), str(download_dir),
                prefix = 'rgb_' + provName + '_' + str(yr) + '_' + str(local_idx) + '_',
                crs = "EPSG:4326", scale = 10)
            time.sleep(100)

def parallelize_download(func, argument_list, num_processes):
 
    pool = Pool(processes=num_processes)
 
    jobs = [pool.apply_async(func=func, args=(*argument,)) 
            if isinstance(argument, tuple) 
            else pool.apply_async(func=func, args=(argument,)) 
            for argument in argument_list]
    pool.close()
    result_list_tqdm = []
    for job in tqdm(jobs):
        result_list_tqdm.append(job.get())
        # Add a wait of 1 second
        time.sleep(180)
 
    return result_list_tqdm

In [5]:
# Create argument
argument_list_config_all = [(yrList, out_dir, provName, str(local_idx),
                             asset_path, selectProv, list_roi_all.get(local_idx)
                             ) for local_idx in range(grid_size)]

5. Download RGB 
    * Single tile Download
    * Sequential Download
    * Parallel Download

In [ ]:
# Single tile Download

yrList, download_dir, provName, local_idx, asset_path, selectProv, tile_shp = argument_list_config_all[10]
argument_list_config_all[10]

get_crp_rgb_from_asset(yrList, out_dir, provName, local_idx, asset_path, selectProv, tile_shp)

([2021, 2022, 2023, 2024],
 'c:\\Users\\spn733\\Work\\CSA_Field_Boundary_Segmentation_V2\\CSA_Field_Boundary_Segmentation\\5_Data\\RGB_download\\Saskatchewan',
 'SK',
 '10',
 'projects/just-amp-296821/assets/road_sk_utm13',
 <ee.computedobject.ComputedObject at 0x1fc1b8644f0>)

In [10]:
# Sequential Download

tiles = [10, 11, 12]
for tile in tiles:
    yrList, download_dir, provName, local_idx, asset_path, selectProv, tile_shp = argument_list_config_all[tile]
    get_crp_rgb_from_asset(yrList, out_dir, provName, local_idx, asset_path, selectProv, tile_shp)


Crop RGB exists for file  rgb_SK_2021_10_1 , skipping now ...
Crop RGB exists for file  rgb_SK_2022_10_1 , skipping now ...
Crop RGB exists for file  rgb_SK_2022_10_1 , skipping now ...
Crop RGB exists for file  rgb_SK_2023_10_1 , skipping now ...
Crop RGB exists for file  rgb_SK_2023_10_1 , skipping now ...
Crop RGB exists for file  rgb_SK_2024_10_1 , skipping now ...
Crop RGB exists for file  rgb_SK_2024_10_1 , skipping now ...


rgb_SK_2021_11_1.tif: |          | 0.00/75.0M (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 1 tiles in 120.49307084083557 seconds.


rgb_SK_2022_11_1.tif: |          | 0.00/75.0M (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 1 tiles in 135.91017818450928 seconds.


...

Downloaded 1 tiles in 169.6560661792755 seconds.


rgb_SK_2023_12_1.tif: |          | 0.00/75.0M (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 1 tiles in 234.7254331111908 seconds.


rgb_SK_2024_12_1.tif: |          | 0.00/75.0M (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 1 tiles in 190.98084712028503 seconds.


In [ ]:
# # Parallelize rgb download

# result_list = parallelize_download(func = get_crp_rgb_from_asset,
#                                    argument_list = argument_list_config_all,
#                                    num_processes = cpus_)